In [36]:
# https: //json-schema.org/understanding-json-schema/index.html
# https: //python-jsonschema.readthedocs.io/en/stable/
from jsonschema import validate

In [37]:
schema = {
    "properties": {
        "name": {
            "type": "string",
            "minLength": 2,
            "maxLength": 400
        },
        "gender": {
            "type": "string",
            "enum": [
                "Male",
                "Female",
                "Other"
            ]
        },
        "age": {
            "type": "number",
            "minimum": 18,
            "maximum": 126
        },
        "dues": {
            "type": "number"
        },
        "address": {
            "type": "string",
            "minLength": 0,
            "maxLength": 1000
        },
        "phones": {
            "properties": {
                "home": {
                    "type": "string"
                },
                "office": {
                    "type": "string"
                }
            },

        },

    },
}

In [40]:
post_data = {
    "name": "Ankur Gupta",
    "age": 36,
    "gender": "Male",
    "dues": 29.83,
    "address": "Home sweet Home",
    "phones": {
        "home": "9099999999",
        "office": "3838383838"
    }
}
validate(post_data, schema)

In [6]:
from cerberus import Validator, schema
# http://docs.python-cerberus.org/en/stable/index.html

In [41]:
# basic example
schema = {'name': {'type': 'string'}, 'age': {'type': 'integer', 'min': 10}}
document = {'name': 'Little Joe', 'age': 5}
v = Validator()
v.validate(document, schema)

False

In [42]:
v.errors

{'age': ['min value is 10']}

In [43]:
# rename fields
v = Validator({'foo': {'rename': 'bar'}})
v.normalized({'foo': 0})

{'bar': 0}

In [44]:
# lamda to evaluate values
even_digits = lambda x: '0' + x if len(x) % 2 else x
v = Validator({}, allow_unknown={'rename_handler': [str, even_digits]})
v.normalized({2: 'foo'})

{'02': 'foo'}

In [45]:
# Purging Unknown Fields
v = Validator({'foo': {'type': 'string'}}, purge_unknown=True)
v.normalized({'bar': 'foo'})

{}

In [46]:
# default values
v.schema = {'amount': {'type': 'integer'}, 'kind': {'type': 'string', 'default': 'purchase'}}
v.normalized({'amount': 1}) == {'amount': 1, 'kind': 'purchase'}

True

In [35]:
import datetime
import json
import typesystem
# https://www.encode.io/typesystem/
# https://www.encode.io/typesystem/fields/

In [47]:
class Phone(typesystem.Schema):
	home = typesystem.String(max_length=100)
	office = typesystem.String(max_length=100)

class Person(typesystem.Schema):
    name = typesystem.String(max_length=100)
    gender = typesystem.Choice(choices=(("M", "Male"),("F", "Female")))
    age = typesystem.String(max_length=100)
    has_dues = typesystem.Boolean(default=False)
    address = typesystem.String(max_length=100)
    phone_no = typesystem.Reference(Phone)
    created_at = typesystem.Date(default=datetime.date.today)

In [48]:
person = Person.validate({
    "name": "Ankur Gupta",
    "gender": "M",
    "age": "36",
    "dues": False,
    "address": "Home Sweet Home.",
    "phone_no":{"home": "9020200202", "office":"39393993"},
})

In [49]:
person


Person(name='Ankur Gupta', gender='M', age='36', has_dues=False, address='Home Sweet Home.', phone_no=Phone(home='9020200202', office='39393993'), created_at=datetime.date(2019, 3, 16))

In [50]:
person.created_at

datetime.date(2019, 3, 16)

In [51]:
print(person['created_at'])

2019-03-16


In [52]:
dict(person)

{'name': 'Ankur Gupta',
 'gender': 'M',
 'age': '36',
 'has_dues': False,
 'address': 'Home Sweet Home.',
 'phone_no': {'home': '9020200202', 'office': '39393993'},
 'created_at': '2019-03-16'}

In [53]:
typesystem.to_json_schema(Person)

{'type': 'object',
 'properties': {'name': {'type': 'string', 'minLength': 1, 'maxLength': 100},
  'gender': {'enum': ['M', 'F']},
  'age': {'type': 'string', 'minLength': 1, 'maxLength': 100},
  'has_dues': {'type': 'boolean', 'default': False},
  'address': {'type': 'string', 'minLength': 1, 'maxLength': 100},
  'phone_no': {'$ref': '#/definitions/Phone'},
  'created_at': {'type': 'string',
   'default': <function date.today>,
   'minLength': 1,
   'format': 'date'}},
 'required': ['name', 'gender', 'age', 'address', 'phone_no'],
 'definitions': {'Phone': {'type': 'object',
   'properties': {'home': {'type': 'string', 'minLength': 1, 'maxLength': 100},
    'office': {'type': 'string', 'minLength': 1, 'maxLength': 100}},
   'required': ['home', 'office']}}}